In [464]:
aa_map = pd.read_csv('../src_data/aa_chart.csv', index_col=None)
str(list(aa_map['Function']))
AA = list('FYWAVILMSTNQPGDEKHRC')
NUM = [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 5, 6, 6, 6, 7]
EIIP = [0.0946, 0.0516, 0.0548, 0.0373, 0.0057, 0.0, 0.0, 0.0823,\
                     0.0829, 0.0941, 0.0036, 0.0761, 0.0198, 0.005, 0.1263, 0.0058, \
                     0.0371, 0.0242, 0.0959, 0.0829]
FNS = ['Aromatic', 'Aromatic', 'Aromatic', 'Hydrophobic', 'Hydrophobic', \
                    'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Polar', 'Polar', 'Polar', \
                    'Polar', 'Proline', 'Glycine', 'Charge (-)', 'Charge (-)', 'Charge (+)', \
                    'Charge (+)', 'Charge (+)', 'Excluded']
AA_map = dict.fromkeys(["NUM", "EIIP", "FNS"])
for conv in list(AA_map.keys()):
    AA_map[conv] = dict(zip(AA, EIIP))
# ---- helper lambda functions
AA_conv = lambda typ, pep: [AA_map[typ][AA] if AA in AA else 0 for AA in pep]

print(AA_map)
print(list(AA_map.keys()))
for p in grbp5_sim.pep_data['Sequences']:
    pass
print(grbp5_sim.data['BLOSUM']['A']['B'])

{'NUM': {'F': 0.0946, 'Y': 0.0516, 'W': 0.0548, 'A': 0.0373, 'V': 0.0057, 'I': 0.0, 'L': 0.0, 'M': 0.0823, 'S': 0.0829, 'T': 0.0941, 'N': 0.0036, 'Q': 0.0761, 'P': 0.0198, 'G': 0.005, 'D': 0.1263, 'E': 0.0058, 'K': 0.0371, 'H': 0.0242, 'R': 0.0959, 'C': 0.0829}, 'EIIP': {'F': 0.0946, 'Y': 0.0516, 'W': 0.0548, 'A': 0.0373, 'V': 0.0057, 'I': 0.0, 'L': 0.0, 'M': 0.0823, 'S': 0.0829, 'T': 0.0941, 'N': 0.0036, 'Q': 0.0761, 'P': 0.0198, 'G': 0.005, 'D': 0.1263, 'E': 0.0058, 'K': 0.0371, 'H': 0.0242, 'R': 0.0959, 'C': 0.0829}, 'FNS': {'F': 0.0946, 'Y': 0.0516, 'W': 0.0548, 'A': 0.0373, 'V': 0.0057, 'I': 0.0, 'L': 0.0, 'M': 0.0823, 'S': 0.0829, 'T': 0.0941, 'N': 0.0036, 'Q': 0.0761, 'P': 0.0198, 'G': 0.005, 'D': 0.1263, 'E': 0.0058, 'K': 0.0371, 'H': 0.0242, 'R': 0.0959, 'C': 0.0829}}
['NUM', 'EIIP', 'FNS']
-2


In [494]:
# @TODO Dynamically filter peptide set based on length(s) of input sequences of binders
#       i.e. 2 binders, one 11 AA long, one 13 AA long, each gets their own "subset" of the
#       full peptide lilst that can be compared to it. For any number of input sequences

# @TODO implement method for both similarities to M6 and GrBP5 to interrelate and act as their own feature set:
# i.e. if a peptide matches both peptides in temrs of sequence at some index, that should be important rather than
# having it equal to one matching only one

# @TODO: Maybe as originally planned implement binders inputted as dictionary of multiple values, each with separate
#      dataframes within the same class --> for big similarity calculations. Or just create multiple SequenceSimilarity instances

'''
Class to calculate several simialrity metrics for an input list of peptides given a sequence string to compare it to.
USAGE: 
1. Create SequenceSimilarityObject({sequence string}, {dictionary of data paths} (to be deprecated soon -- use
   internal class data), {path to peptides csv}, {column title of sequence values of aforementioned peptides csv})
   for any number of sequences you want to be compared to the list of peptides
2. For each object, call object.generate_similarity() to fill out the Dataframe with similarity metrics
3. To whittle down this similarity matrix to list only those peptides with pattern matching of a minimum length
   at a matching index in the rerence peptide (henceforth the binder), call object.get_df_with_binder_subseqs(min_length={#})
@ Author: Chris Pecunies, with help from Savvy Gupta and Aaron Tsang
@ Date: February 12, 2020
'''
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
from typing import Set, Tuple, Dict, List
from scipy import interpolate

class SequenceSimilarity:
    '''
    Class that takes in a path to a list of amino acid sequences as well
    as any number of peptide sequences explicitly that are known to have
    a certain set of properties. Generates metrics for similarity for each
    peptide in path and returns domains AA sequence with high similarity
    '''

    def __init__(self, binder: str,   
                 data_paths: Dict,     #@TODO Make another .py file containing object version of necessary sim matrices/conversions, etc.
                 peps_path: str,       #      to reduce reliance on outside data
                 aa_col: str):         #-> The column in peps_path csv where sequences are held
        
        # ---- setting data ---------
        self.AA = list('FYWAVILMSTNQPGDEKHRC')
        self.conv = ["NUM", "EIIP", "FNS"]
        self.con_vals = dict.fromkeys(self.conv)
        self.con_vals['NUM'] = [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 5, 6, 6, 6, 7]
        self.con_vals['EIIP'] = [0.0946, 0.0516, 0.0548, 0.0373, 0.0057, 0.0, 0.0, 0.0823,\
                     0.0829, 0.0941, 0.0036, 0.0761, 0.0198, 0.005, 0.1263, 0.0058, \
                     0.0371, 0.0242, 0.0959, 0.0829]
        self.con_vals['FNS'] = ['Aromatic', 'Aromatic', 'Aromatic', 'Hydrophobic', 'Hydrophobic', \
                    'Hydrophobic', 'Hydrophobic', 'Hydrophobic', 'Polar', 'Polar', 'Polar', \
                    'Polar', 'Proline', 'Glycine', 'Charge (-)', 'Charge (-)', 'Charge (+)', \
                    'Charge (+)', 'Charge (+)', 'Excluded']
        self.AA_map = dict.fromkeys(self.conv)
        for conv in self.conv:
            self.AA_map[conv] = dict(zip(self.AA, self.con_vals[conv]))
        self.binder = binder   # to get binder_len, just use len(self.binders)
        self.__read_similarity_data(data_paths) # !TO BE DEPRECATED! Just store data in class
        
        # ---- helper lambda functions
        self.AA_conv = lambda typ, pep: [self.AA_map[typ][AA] if AA in self.AA else 0 for AA in pep]
        self.get_sim = lambda p1, p2, t: sum([self.data[t][a1][a2] for a1 in p1 for a2 in p2])
        
        # ---- setting up dataframes
        self.aa_col = aa_col
        self.columns = ['EIIP_Seq', 'NUM_Seq', 'PAM30', 'BLOSUM', 'RRM_SN', 'RRM_Corr', 'FNS_Seq']
        ## @TODO: Add "# of matching sseqs, cross entropy AA, cross entropy Num columns ?"
        
        self.peps = pd.read_csv(peps_path)
        self.peps = self.peps[~self.peps[self.aa_col].str.contains("O")]
        self.peps.columns = [aa_col]
        self.peps_same_len = self.peps[self.peps[aa_col].str.len() == len(binder)]
        if len(self.peps_same_len) == 0:
            raise Exception("No peptides of same length as binder found")
        self.pep_data = self.peps_same_len.copy()
        for col in self.columns:
            self.pep_data[col] = None
        
        #self.get_similarities()

    def __read_similarity_data(self, data_path_dict) -> None:
        """
        Private method to store the paths of any data needed
        for similarity calcs and create Dataframes from them
        """
        self.data = dict.fromkeys(data_path_dict.keys())
        for data in self.data.keys():
            self.data[data] = pd.read_csv(data_path_dict[data], index_col=0)
            self.data[data] = self.data[data].to_dict()
            
    def _update_AA_conversion(self, conv_type: str = None) -> None:
        """
        Adds to the initially empty column values for conv_type (possible choices
        'EIIP' and 'Num' for now) the conversion of the AA sequence in the self.aa_col
        column a list representing its conversion
        """
        seqs = self.pep_data[self.aa_col]
        self.pep_data[conv_type+"_Seq"] = [self.AA_conv(conv_type, p) for p in seqs]

    
    def _update_matrix_similarity(self) -> None:
        """
        Just updates the similarity columns for the output similarity dataframe.
        Uses lambda helper function self.get_sim in __init__
        """
        for data in self.data.keys():
            sim = [self.get_sim(p, self.binder, data) for p in self.pep_data[self.aa_col]]
            self.pep_data[data] = np.interp(sim, (min(sim), max(sim)), (0,100))
        

    def _update_RRM_similarity(self) -> None:
        """
        Uses the Resonant Recognition Model as described by Irena Cosic to 
        """
        get_dft_from_eiip = lambda eiip: np.fft.rfft(eiip)[1:]
        get_cross_spectrum = lambda p1, p2: [x1*x2 for x1, x2 in zip(p1, p2)]

    def _update_RRM_corr_similarity(self) -> None:
        pass

    
    # NOTE! Adds columns "Matching_sseqs" and "Num_matching" to output
    # Might be too unwieldy / unhelpful for output similarity data
    # if so, just comment out _update_matching_sseqs()
    def _update_matching_sseqs(self, single_match_weight:float = 0.5, weight:float = 2) -> None:
        """
        Returns a number as a new column representing the number of "matches" a peptide
        has for all possible subsequences for the binder inputted at a given index. For
        weighting=1, all matches are treated equally ('Y' at position 3 is treated equal
        to IMV at position 0) but lowering weighting lowers smaller-length matches
        """
        # @TODO Remove "duplicates" which occur at different matching indexes of binder
        # but are part of a larger pattern already recorded at an earlier index
        self.pep_data['sseq_matches'] = None
        self.pep_data['weighted_matches'] = None
        
        score = lambda s: (single_match_weight * 1) + (len(s)**weight)
        all_sseqs = self.get_binder_subseq()
        matches = list(); num_matches = list()
        for i, seq in enumerate(self.pep_data[self.aa_col]):
            matches.append([]); num_matches.append(0)
            longest_match = None
            for j, AA in enumerate(seq): #-----------goes thru each sequence
                prev_match: Tuple = None
                for k, (sseq, bin_i) in enumerate(all_sseqs): #---------for each AA in seq, then go through all sseq and indexes
                    if (bin_i == j) and (seq[j:len(sseq)+j] == sseq): #--- if AA matches sseq and index of pep == index of sseq:
                        if longest_match is not None:
                            if seq[longest_match[1]:longest_match[1]+longest_match[2]].find(seq[j:len(sseq)+j]) >= 0:
                                continue
                        prev_binder_i: int = all_sseqs[k-1][1]
                        prev_binder_sseq: str = all_sseqs[k-1][0]
                        if prev_binder_i == bin_i:
                            matches[i].pop()
                            num_matches[i] -= score(prev_match[0])
                            longest_match = (sseq, bin_i, len(sseq))
                        matches[i].append((sseq, bin_i))   
                        num_matches[i] += score(sseq)
                        prev_match = (sseq, bin_i)
        
        self.pep_data['sseq_matches'] = matches
        self.pep_data['weighted_matches'] = np.interp(num_matches, (min(num_matches), max(num_matches)), (0,100))
        
                    
    def remove_matching_sseqs_column(self) -> None:
        """
        Just removes the binder sseq pattern matches list column and number
        of matching (with/without) weighting if they exist
        """
        if self.pep_data.columns.contains(['sseq_matches', 'num_matches']):
            self.pep_data = self.pep_data.drop(columns=['sseq_matches', 'num_matches'])
                
    
    def update_similarities(self) -> None:
        '''
        Updates the similarity values whenever called (for now should be only once right
        after creating the object, ecept possibly if the Binding peptide is updated
        (should be handled automatically)
        '''
        self._update_AA_conversion(conv_type='EIIP')
        self._update_AA_conversion(conv_type='NUM')
        self._update_matrix_similarity()
        self._update_RRM_SN_similarity()
        self._update_RRM_corr_similarity()
        self._update_matching_sseqs()

    def df_filter_subseq(self, sub_seq: str, ind: int = None) -> pd.DataFrame:
        '''
        Takes in a subsequence of equal or lesser length to
        peptides in class peptide dataframe and returns a dataframe
        containing only those peptides containing the sequence
        '''
        if not {*sub_seq}.issubset({*self.AA}):
            raise Exception('Invalid subsequence')
        if ind is None:
            return self.pep_data[self.pep_data[self.aa_col].str.contains(sub_seq)]
        return self.pep_data[self.pep_data[self.aa_col].str.find(sub_seq) == ind]

    def get_sim_matrix(self, seq) -> pd.DataFrame:
        return self.data.filter

    def get_binder_subseq(self) -> pd.DataFrame: #Each binder associated with list of (sseq, index)
        '''
        Generates all possible subsequences for binders. Returns ia list of tuples, where
        each entry of the list is of the form ({Sub Seq}, {Ind}) of binder
        '''
        self.sseqs = [(self.binder[i:j], i) for i in range(len(self.binder)) for j in range(i+1, len(self.binder)+1)]
        return self.sseqs


    def get_df_with_binder_subseqs(self, min_length: int = 0) -> Dict[str, pd.DataFrame]:
        '''
        Returns a filtered version of self.peps_same_len DataFrame containing only
        those rows with sequences which contain subsequences (of min length specified in parameter) 
        of the two binder sequences in the locations where they occur in the binders
        '''
        sseq = self.get_binder_subseq()
        filtered_data = [self.df_filter_subseq(ss,i) for (ss,i) in sseq if len(ss) >= min_length]
        return pd.concat(filtered_data)
    
    #------------------helper methods ------------------------------------#
    
    def aa_dict(self, conversion) -> Dict:
        if conversion == 'NUM':
            return dict(zip(self.AA, self.NUM))
        elif conversion == 'EIIP':
            return dict(zip(self.AA, self.EIIP))
        elif conversion == 'FNS':
            return dict(zip(self.AA, self.FNS))
        else:
            raise Exception("Must specify AA conversion target")
                
                
    #-------------------miscellaneous methods------------------------------#

    def get_kendalltau_corr_map(self) -> Tuple:
        return kendalltau(self.data['AA_MAP'][['Num']], self.data['AA_MAP'][['EIIP']])
    
'''
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform


def similarity_func(u, v):
    return 1/(1+euclidean(u,v))

DF_var = pd.DataFrame.from_dict({"s1":[1.2,3.4,10.2],"s2":[1.4,3.1,10.7],"s3":[2.1,3.7,11.3],"s4":[1.5,3.2,10.9]})
DF_var.index = ["g1","g2","g3"]

dists = pdist(DF_var, similarity_func)
DF_euclid = pd.DataFrame(squareform(dists), columns=DF_var.index, index=DF_var.index)
'''

'\nimport pandas as pd\nfrom scipy.spatial.distance import euclidean, pdist, squareform\n\n\ndef similarity_func(u, v):\n    return 1/(1+euclidean(u,v))\n\nDF_var = pd.DataFrame.from_dict({"s1":[1.2,3.4,10.2],"s2":[1.4,3.1,10.7],"s3":[2.1,3.7,11.3],"s4":[1.5,3.2,10.9]})\nDF_var.index = ["g1","g2","g3"]\n\ndists = pdist(DF_var, similarity_func)\nDF_euclid = pd.DataFrame(squareform(dists), columns=DF_var.index, index=DF_var.index)\n'

In [495]:

DATA_PATHS = {
    "BLOSUM":"../src_data/BLOSUM.csv",
    "PAM30":"../src_data/pam30.csv",
}
SEQS = {
    'GRBP5':'IMVTESSDYSSY',
    'M6':'IMVTASSAYDDY'
}
AA_COL = 'Sequences'
PEP_PATH = '../src_data/Sequence_data.csv' 
grbp5_sim = SequenceSimilarity(SEQS['GRBP5'], DATA_PATHS, PEP_PATH, AA_COL)
m6_sim = SequenceSimilarity(SEQS['M6'], DATA_PATHS, PEP_PATH, AA_COL)
grbp5_sim.update_similarities()
m6_sim.update_similarities()
grbp5_sim.pep_data.to_csv('../out/grbp5_almost_final.csv')
m6_sim.pep_data.to_csv('../out/m6_almost_final.csv')

AttributeError: 'SequenceSimilarity' object has no attribute '_update_RRM_SN_similarity'

In [474]:
print(m6_sim.sseqs)
m6_ss = m6_sim.get_df_with_binder_subseqs()
grb_ss = grbp5_sim.get_df_with_binder_subseqs()
grb_ss[grb_ss['sseq_matches'].str.len() > 2]

[('I', 0), ('IM', 0), ('IMV', 0), ('IMVT', 0), ('IMVTA', 0), ('IMVTAS', 0), ('IMVTASS', 0), ('IMVTASSA', 0), ('IMVTASSAY', 0), ('IMVTASSAYD', 0), ('IMVTASSAYDD', 0), ('IMVTASSAYDDY', 0), ('M', 1), ('MV', 1), ('MVT', 1), ('MVTA', 1), ('MVTAS', 1), ('MVTASS', 1), ('MVTASSA', 1), ('MVTASSAY', 1), ('MVTASSAYD', 1), ('MVTASSAYDD', 1), ('MVTASSAYDDY', 1), ('V', 2), ('VT', 2), ('VTA', 2), ('VTAS', 2), ('VTASS', 2), ('VTASSA', 2), ('VTASSAY', 2), ('VTASSAYD', 2), ('VTASSAYDD', 2), ('VTASSAYDDY', 2), ('T', 3), ('TA', 3), ('TAS', 3), ('TASS', 3), ('TASSA', 3), ('TASSAY', 3), ('TASSAYD', 3), ('TASSAYDD', 3), ('TASSAYDDY', 3), ('A', 4), ('AS', 4), ('ASS', 4), ('ASSA', 4), ('ASSAY', 4), ('ASSAYD', 4), ('ASSAYDD', 4), ('ASSAYDDY', 4), ('S', 5), ('SS', 5), ('SSA', 5), ('SSAY', 5), ('SSAYD', 5), ('SSAYDD', 5), ('SSAYDDY', 5), ('S', 6), ('SA', 6), ('SAY', 6), ('SAYD', 6), ('SAYDD', 6), ('SAYDDY', 6), ('A', 7), ('AY', 7), ('AYD', 7), ('AYDD', 7), ('AYDDY', 7), ('Y', 8), ('YD', 8), ('YDD', 8), ('YDDY', 8

,Sequences,EIIP_Seq,NUM_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr,FNS_Seq,sseq_matches,weighted_matches
8974,IGDEENSAKFPI,"[0.0, 0.005, 0.1263, 0.0058, 0.0058, 0.0036, 0...","[1, 4, 5, 5, 5, 2, 2, 1, 6, 0, 3, 1]",66.816143,58.895706,None,None,None,"[(I, 0), (E, 4), (S, 6)]",47.368421
8992,IGDEENSAKFPI,"[0.0, 0.005, 0.1263, 0.0058, 0.0058, 0.0036, 0...","[1, 4, 5, 5, 5, 2, 2, 1, 6, 0, 3, 1]",66.816143,58.895706,None,None,None,"[(I, 0), (E, 4), (S, 6)]",47.368421
9007,IGDEENSAKFPI,"[0.0, 0.005, 0.1263, 0.0058, 0.0058, 0.0036, 0...","[1, 4, 5, 5, 5, 2, 2, 1, 6, 0, 3, 1]",66.816143,58.895706,None,None,None,"[(I, 0), (E, 4), (S, 6)]",47.368421
9022,IGDEENSAKFPI,"[0.0, 0.005, 0.1263, 0.0058, 0.0058, 0.0036, 0...","[1, 4, 5, 5, 5, 2, 2, 1, 6, 0, 3, 1]",66.816143,58.895706,None,None,None,"[(I, 0), (E, 4), (S, 6)]",47.368421
9322,IGDEENSAKFPI,"[0.0, 0.005, 0.1263, 0.0058, 0.0058, 0.0036, 0...","[1, 4, 5, 5, 5, 2, 2, 1, 6, 0, 3, 1]",66.816143,58.895706,None,None,None,"[(I, 0), (E, 4), (S, 6)]",47.368421
...,...,...,...,...,...,...,...,...,...,...
4249,DRVAELDQLLHY,"[0.1263, 0.0959, 0.0057, 0.0373, 0.0058, 0.0, ...","[5, 6, 1, 1, 5, 1, 5, 2, 1, 1, 6, 0]",35.201794,53.987730,None,None,None,"[(V, 2), (E, 4), (Y, 11)]",47.368421
12865,LDLTPGSHVDSY,"[0.0, 0.1263, 0.0, 0.0941, 0.0198, 0.005, 0.08...","[1, 5, 1, 2, 3, 4, 2, 6, 1, 5, 2, 0]",56.726457,68.098160,None,None,None,"[(T, 3), (S, 6), (SY, 10)]",78.947368
12901,LTLTPGSHVDSY,"[0.0, 0.0941, 0.0, 0.0941, 0.0198, 0.005, 0.08...","[1, 2, 1, 2, 3, 4, 2, 6, 1, 5, 2, 0]",64.349776,73.619632,None,None,None,"[(T, 3), (S, 6), (SY, 10)]",78.947368
25345,SVAFERSAMQNY,"[0.0829, 0.0057, 0.0373, 0.0946, 0.0058, 0.095...","[2, 1, 1, 0, 5, 6, 2, 1, 1, 2, 2, 0]",78.251121,87.116564,None,None,None,"[(E, 4), (S, 6), (Y, 11)]",47.368421


In [477]:
GRBP5 = 'IMVTESSDYSSY'
M6 = 'IMVTASSAYDDY'
print(len(m6_sim.sseqs))
m6_sim.pep_data.sort_values(by='BLOSUM', ascending=False)
grbp5_sim.pep_data.sort_values(by='PAM30', ascending=False)

78


,Sequences,EIIP_Seq,NUM_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr,FNS_Seq,sseq_matches,weighted_matches
3434,SAKVAFSATRST,"[0.0829, 0.0373, 0.0371, 0.0057, 0.0373, 0.094...","[2, 1, 6, 1, 1, 0, 2, 1, 2, 6, 2, 2]",100.000000,100.000000,None,None,None,"[(S, 6), (S, 10)]",31.578947
26114,TAAAETETRTHT,"[0.0941, 0.0373, 0.0373, 0.0373, 0.0058, 0.094...","[2, 1, 1, 1, 5, 2, 5, 2, 6, 2, 6, 2]",97.982063,92.024540,None,None,None,"[(E, 4)]",15.789474
28666,TAAAETETRTHT,"[0.0941, 0.0373, 0.0373, 0.0373, 0.0058, 0.094...","[2, 1, 1, 1, 5, 2, 5, 2, 6, 2, 6, 2]",97.982063,92.024540,None,None,None,"[(E, 4)]",15.789474
2809,DASPSATSGNHS,"[0.1263, 0.0373, 0.0829, 0.0198, 0.0829, 0.037...","[5, 1, 2, 3, 2, 1, 2, 2, 4, 2, 6, 2]",97.757848,95.092025,None,None,None,[],0.000000
2807,SFDASPSATSGN,"[0.0829, 0.0946, 0.1263, 0.0373, 0.0829, 0.019...","[2, 0, 5, 1, 2, 3, 2, 1, 2, 2, 4, 2]",97.533632,95.092025,None,None,None,"[(S, 6), (S, 9)]",31.578947
...,...,...,...,...,...,...,...,...,...,...
14889,GFLRRIRPKLKW,"[0.005, 0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.09...","[4, 0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0]",5.156951,7.361963,None,None,None,[],0.000000
15058,GFLRRIRPKLKW,"[0.005, 0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.09...","[4, 0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0]",5.156951,7.361963,None,None,None,[],0.000000
28283,GFLRRIRPKLKW,"[0.005, 0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.09...","[4, 0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0]",5.156951,7.361963,None,None,None,[],0.000000
14720,GFLRRIRPKLKW,"[0.005, 0.0946, 0.0, 0.0959, 0.0959, 0.0, 0.09...","[4, 0, 1, 6, 6, 1, 6, 3, 6, 1, 6, 0]",5.156951,7.361963,None,None,None,[],0.000000


In [405]:
m6_sim.get_df_with_binder_subseqs()
grbp5_sim.get_df_with_binder_subseqs()

,Sequences,EIIP_Seq,NUM_Seq,PAM30,BLOSUM,RRM_SN,RRM_Corr,sseq_matches,weighted_matches
400,IRYHQCYFNPIS,"[0.0, 0.0959, 0.0516, 0.0242, 0.0761, 0.0829, ...","[1, 6, 0, 6, 2, 7, 0, 0, 2, 3, 1, 2]",None,None,None,None,"[(I, 0)]",15.789474
584,IRYHQCYFNPIS,"[0.0, 0.0959, 0.0516, 0.0242, 0.0761, 0.0829, ...","[1, 6, 0, 6, 2, 7, 0, 0, 2, 3, 1, 2]",None,None,None,None,"[(I, 0)]",15.789474
615,IRYRQCYFNPIS,"[0.0, 0.0959, 0.0516, 0.0959, 0.0761, 0.0829, ...","[1, 6, 0, 6, 2, 7, 0, 0, 2, 3, 1, 2]",None,None,None,None,"[(I, 0)]",15.789474
1081,IRYHQCYFNPIS,"[0.0, 0.0959, 0.0516, 0.0242, 0.0761, 0.0829, ...","[1, 6, 0, 6, 2, 7, 0, 0, 2, 3, 1, 2]",None,None,None,None,"[(I, 0)]",15.789474
1888,INDDCPNLIGNR,"[0.0, 0.0036, 0.1263, 0.1263, 0.0829, 0.0198, ...","[1, 2, 5, 5, 7, 3, 2, 1, 1, 4, 2, 6]",None,None,None,None,"[(I, 0)]",15.789474
...,...,...,...,...,...,...,...,...,...
27481,QGPWLEEEEEAY,"[0.0761, 0.005, 0.0198, 0.0548, 0.0, 0.0058, 0...","[2, 4, 3, 0, 1, 5, 5, 5, 5, 5, 1, 0]",None,None,None,None,"[(Y, 11)]",15.789474
27899,ERPRMEEEEEAY,"[0.0058, 0.0959, 0.0198, 0.0959, 0.0823, 0.005...","[5, 6, 3, 6, 1, 5, 5, 5, 5, 5, 1, 0]",None,None,None,None,"[(Y, 11)]",15.789474
28099,QHGAAPECFWKY,"[0.0761, 0.0242, 0.005, 0.0373, 0.0373, 0.0198...","[2, 6, 4, 1, 1, 3, 5, 7, 0, 0, 6, 0]",None,None,None,None,"[(Y, 11)]",15.789474
28621,QHGTAPECFWKY,"[0.0761, 0.0242, 0.005, 0.0941, 0.0373, 0.0198...","[2, 6, 4, 2, 1, 3, 5, 7, 0, 0, 6, 0]",None,None,None,None,"[(T, 3), (Y, 11)]",31.578947


In [404]:
print(type(similarity.peps))
peps = pd.read_csv('../src_data/Sequence_data.csv')
peps.columns = ['Sequences']

peps

<class 'dict'>


,Sequences
0,FSEEEKEPE
1,SVPHFSDED
2,VPHFSDEDK
3,PHFSDEDKD
4,HFSDEDKDP
...,...
28910,VMGHFRWDR
28911,MGHFRWDRF
28912,YVMGHFRWDR
28913,VMGHFRWDRF


# Get RRM S/N similarity

In [12]:
def get_RRM():
    data = similarity.pep_data
    matrix = similarity.data['PA]
    print(data.head(), len(data))
    
get_RRM()

       Sequences                                           EIIP_Seq  \
13  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
14  VPHFSDEDKDPE  [0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.126...   
28  SVPHFSEEEKEA  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   
29  VPHFSEEEKEAE  [0.0057, 0.0198, 0.0242, 0.0946, 0.0829, 0.005...   
43  SVPHFSDEDKDP  [0.0829, 0.0057, 0.0198, 0.0242, 0.0946, 0.082...   

                                 Num_Seq PAM30 BLOSUM RRM_SN RRM_Corr  
13  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]  None   None   None     None  
14  [1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3, 5]  None   None   None     None  
28  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1]  None   None   None     None  
29  [1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 1, 5]  None   None   None     None  
43  [2, 1, 3, 6, 0, 2, 5, 5, 5, 6, 5, 3]  None   None   None     None   3079


# Get BLOSUM similarity

In [29]:
def get_BLOSUM_similarity():
    data = similarity.pep_data
    binders = similarity.binders
    BLOSUM = similarity.data['BLOSUM']
    for binder in binders:
        for pep in data['Sequences']:
            dist = 0
            for aa1 in pep:
                for aa2 in binder:
                    dist += BLOSUM[aa1].loc[aa2]
                    
    print(data['Sequences'])
    print(binders)
    print(BLOSUM)
    
#get_BLOSUM_similarity()
similarity.data['BLOSUM']

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4
Q,-1,1,0,0,-3,5,2,-2,0,-3,...,-1,0,-1,-2,-1,-2,0,3,-1,-4
E,-1,0,0,2,-4,2,5,-2,0,-3,...,-1,0,-1,-3,-2,-2,1,4,-1,-4
G,0,-2,0,-1,-3,-2,-2,6,-2,-4,...,-2,0,-2,-2,-3,-3,-1,-2,-1,-4
H,-2,0,1,-1,-3,0,0,-2,8,-3,...,-2,-1,-2,-2,2,-3,0,0,-1,-4
I,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,...,-3,-2,-1,-3,-1,3,-3,-3,-1,-4


# Get PAM30 similiarity

# REVELATION
What if... maybe... instead of handling binder as a dict, handle AT MOST one input string as the binder sequence so there is a one:one correspondence between the class and the sequence for similarity calculations???